In [1]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
import numpy as np

C:\Users\Anneke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

In [4]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [5]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df = 100)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [6]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

In [7]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [8]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [9]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones')(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [10]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [11]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [12]:
y_train_agreement.shape

(25000, 83)

In [13]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=10)

Train on 16667 samples, validate on 8333 samples
Epoch 1/10
16667/16667 [==============================] - 412s 25ms/step - loss: 0.4592 - acc: 0.6924 - val_loss: 0.4030 - val_acc: 0.7818
Epoch 2/10
16667/16667 [==============================] - 100s 6ms/step - loss: 0.3222 - acc: 0.8297 - val_loss: 0.4137 - val_acc: 0.8022
Epoch 3/10
16667/16667 [==============================] - 98s 6ms/step - loss: 0.2829 - acc: 0.8588 - val_loss: 0.4313 - val_acc: 0.8090
Epoch 4/10
16667/16667 [==============================] - 97s 6ms/step - loss: 0.2517 - acc: 0.8752 - val_loss: 0.4390 - val_acc: 0.8192
Epoch 5/10
16667/16667 [==============================] - 106s 6ms/step - loss: 0.2333 - acc: 0.8896 - val_loss: 0.4451 - val_acc: 0.8200s: 0.2328
Epoch 6/10
16667/16667 [==============================] - 106s 6ms/step - loss: 0.2221 - acc: 0.8985 - val_loss: 0.4500 - val_acc: 0.8242
Epoch 7/10
16667/16667 [==============================] - 106s 6ms/step - loss: 0.2053 - acc: 0.9069 - val_loss: 0.

In [14]:
combined_model_history = combined_model.fit([X_train,y_train_agreement], y_train_original, batch_size=1, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 1082s 43ms/step - loss: 0.7383 - acc: 0.3372
Epoch 2/10
25000/25000 [==============================] - 1075s 43ms/step - loss: 0.6829 - acc: 0.3721
Epoch 3/10
25000/25000 [==============================] - 1058s 42ms/step - loss: 0.6784 - acc: 0.3726
Epoch 4/10
25000/25000 [==============================] - 1057s 42ms/step - loss: 0.6809 - acc: 0.3693
Epoch 5/10
25000/25000 [==============================] - 1057s 42ms/step - loss: 0.6802 - acc: 0.3745
Epoch 6/10
25000/25000 [==============================] - 1056s 42ms/step - loss: 0.6804 - acc: 0.3741
Epoch 7/10
25000/25000 [==============================] - 1055s 42ms/step - loss: 0.6793 - acc: 0.3724
Epoch 8/10
25000/25000 [==============================] - 1030s 41ms/step - loss: 0.6837 - acc: 0.3683
Epoch 9/10
25000/25000 [==============================] - 1019s 41ms/step - loss: 0.6815 - acc: 0.3713
Epoch 10/10
25000/25000 [==============================] - 1018s 41ms/ste

In [15]:
# independent weight here
# independent_weight_base_model_history
# independent_weight_combined_model_history

In [16]:
# var name
# shared_weight_base_model_history
# shared_weight_combined_model_history

In [17]:
net_weight = base_model.get_weights()

In [18]:
len(net_weight)

2

In [19]:
weight = net_weight[0].flatten()

In [20]:
net_weight[1].shape

(1,)

In [21]:
words = cv.get_feature_names()

In [22]:
indices = np.argsort(weight)

In [23]:
for i in indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

worst 	 -3.141
incoherent 	 -2.980
dull 	 -2.957
waste 	 -2.531
fails 	 -2.492
poorly 	 -2.477
unwatchable 	 -2.464
awful 	 -2.379
disappointing 	 -2.315
obnoxious 	 -2.292


In [24]:
pos_indices = indices[::-1]
for i in pos_indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

refreshing 	 2.963
excellent 	 2.783
flawless 	 2.729
rare 	 2.651
touching 	 2.564
complaint 	 2.545
captures 	 2.538
wonderfully 	 2.336
underrated 	 2.279
sadness 	 2.206


In [25]:
combined_net_weight = combined_model.get_weights()

In [26]:
len(combined_net_weight)

87

In [27]:
ht_weight = []
for i in range(len(combined_net_weight)):
    if i>1 and i < 85:
        print(combined_net_weight[i].flatten())
        ht_weight.append(combined_net_weight[i].flatten())
    else:
        print(i, combined_net_weight[i].shape)
ht_weight = np.asarray(ht_weight)

0 (3641, 1)
1 (1,)
[3.9346304]
[3.8828895]
[4.0476294]
[4.348806]
[1.9640588]
[1.2679062]
[4.1599655]
[3.9188468]
[3.5799336]
[3.9810755]
[2.9964066]
[4.09745]
[3.7936475]
[4.300665]
[3.3076]
[2.6483078]
[2.931407]
[2.19652]
[2.683866]
[3.8124444]
[4.207136]
[3.7190614]
[3.9740524]
[3.2323294]
[3.8833187]
[3.7204125]
[4.0852666]
[3.8941135]
[2.988484]
[2.986725]
[3.1048138]
[4.303184]
[2.9964528]
[3.4653652]
[3.2079458]
[3.7052884]
[2.7064424]
[3.0926125]
[2.7100933]
[2.7378268]
[2.540685]
[3.9741557]
[3.6661992]
[4.3169155]
[3.9872477]
[3.7749882]
[4.0014734]
[3.7266593]
[2.786077]
[3.8235312]
[3.1753588]
[3.3165832]
[3.8195841]
[3.6359851]
[4.0141053]
[3.393366]
[2.597633]
[4.4814487]
[3.5851068]
[3.3041155]
[3.5707412]
[3.0686507]
[3.1469]
[3.8331268]
[3.8403041]
[3.9222138]
[3.44693]
[3.196657]
[3.8409789]
[3.19162]
[3.2582202]
[3.4272263]
[4.031468]
[3.9488862]
[3.5523536]
[3.0562994]
[4.3331547]
[2.6577194]
[3.3514903]
[3.0067883]
[1.610413]
[3.9973507]
[4.5276856]
85 (83, 1)
86 

In [28]:
sigmoid_weight = combined_net_weight[85].flatten()

In [29]:
ht_weight = ht_weight.flatten()

In [30]:
human_term_indices = np.argsort(ht_weight)

print('human terms \t dense_relu weight \t sigmoid weight \t\t pos/neg?')
for i in human_term_indices:
    print('%s \t %.3f \t %.3f \t\t %.1f' %(word_list[i], ht_weight[i], sigmoid_weight[i], connotation[word_list[i]]))

human terms 	 dense_relu weight 	 sigmoid weight 		 pos/neg?
6/10 	 1.268 	 0.407 		 1.0
wonderfully 	 1.610 	 0.626 		 1.0
5/10 	 1.964 	 1.145 		 1.0
beautifully 	 2.197 	 1.106 		 1.0
great 	 2.541 	 1.730 		 1.0
perfectly 	 2.598 	 1.706 		 1.0
badly 	 2.648 	 -1.821 		 0.0
wasted 	 2.658 	 -1.655 		 0.0
best 	 2.684 	 1.665 		 1.0
fun 	 2.706 	 1.576 		 1.0
funniest 	 2.710 	 1.657 		 1.0
gem 	 2.738 	 1.799 		 1.0
loved 	 2.786 	 1.857 		 1.0
beautiful 	 2.931 	 1.704 		 1.0
enjoyed 	 2.987 	 1.968 		 1.0
enjoyable 	 2.988 	 2.212 		 1.0
amazing 	 2.996 	 1.900 		 1.0
fantastic 	 2.996 	 2.132 		 1.0
wonderful 	 3.007 	 2.075 		 1.0
unfunny 	 3.056 	 -2.155 		 0.0
rare 	 3.069 	 1.941 		 1.0
funny 	 3.093 	 -1.851 		 0.0
excellent 	 3.105 	 2.241 		 1.0
recommended 	 3.147 	 2.065 		 1.0
mess 	 3.175 	 -2.002 		 0.0
subtle 	 3.192 	 2.260 		 1.0
solid 	 3.197 	 2.120 		 1.0
favorite 	 3.208 	 2.255 		 1.0
disappointed 	 3.232 	 -2.373 		 0.0
superb 	 3.258 	 2.197 		 1.0
poorly 	

In [31]:
len(connotation)

83

In [32]:
# Print report on the word transparency
index = [9, 19]
def report():
    for i in index:
        print()
        bm = base_model.predict(X_test[i])
        
        cm = combined_model.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
        print(X_test_original[i], '\n\n actual label : ', y_test_original[i], '\n predict from base model : ', bm.flatten(), '\n predict label : ', cm.flatten())
    
        for i,output in enumerate(concat_output.flatten()):
            if output != 0:
                print(word_list[i], output)

In [33]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [34]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [35]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)

25000/25000 [==============================] - 7s 277us/step


In [36]:
score

[0.1783953401482105, 0.8036]

### Let's see the weight when the trainable is false

In [37]:
def build_combined_model():
    # build the combined model
    # Combined model
    human_terms_len = len(word_list)

    base_model = build_base_model(X_train.shape[1])

    combined_input_layer = Input(shape=(X_train.shape[1],))

    # build the hard coded weight for human terms
    ht_input_layer = Input(shape=(human_terms_len,))

    split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

    # get the document prediction
    label_layer = base_model(combined_input_layer)

    # stack the multiply layer
    dense_layer = []
    for i in range(human_terms_len):
        dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones', trainable=True)(Multiply()([split[i], label_layer])))

    # concat all the result   
    concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

    # pass it to sigmoid layer
    output_layer = Dense(1, activation='sigmoid')(concat)

    combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
    combined_model.summary()
    
    return base_model, combined_model

In [38]:
base_model, combined_model = build_combined_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [39]:
base_model.compile(loss='mse',
                  optimizer='Adagrad',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='Adagrad',
                      metrics=['acc'])

In [40]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=50)

Train on 16667 samples, validate on 8333 samples
Epoch 1/50
16667/16667 [==============================] - 109s 7ms/step - loss: 0.4898 - acc: 0.5327 - val_loss: 0.4250 - val_acc: 0.6006
Epoch 2/50
16667/16667 [==============================] - 109s 7ms/step - loss: 0.3556 - acc: 0.6567 - val_loss: 0.4010 - val_acc: 0.6423
Epoch 3/50
16667/16667 [==============================] - 108s 6ms/step - loss: 0.3237 - acc: 0.6880 - val_loss: 0.3880 - val_acc: 0.6605
Epoch 4/50
16667/16667 [==============================] - 108s 7ms/step - loss: 0.3037 - acc: 0.7055 - val_loss: 0.3820 - val_acc: 0.6699
Epoch 5/50
16667/16667 [==============================] - 109s 7ms/step - loss: 0.2906 - acc: 0.7185 - val_loss: 0.3780 - val_acc: 0.6799
Epoch 6/50
16667/16667 [==============================] - 108s 6ms/step - loss: 0.2801 - acc: 0.7298 - val_loss: 0.3751 - val_acc: 0.6852
Epoch 7/50
16667/16667 [==============================] - 108s 7ms/step - loss: 0.2721 - acc: 0.7362 - val_loss: 0.3747 - v

In [41]:
combined_model_history = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], batch_size=1, epochs=50)

Epoch 1/50
16667/16667 [==============================] - 545s 33ms/step - loss: 0.8861 - acc: 0.2410
Epoch 2/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.7635 - acc: 0.3043
Epoch 3/50
16667/16667 [==============================] - 543s 33ms/step - loss: 0.7291 - acc: 0.3247
Epoch 4/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.7100 - acc: 0.3372
Epoch 5/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.6974 - acc: 0.3452
Epoch 6/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.6879 - acc: 0.3507
Epoch 7/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.6805 - acc: 0.3553
Epoch 8/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.6746 - acc: 0.3592
Epoch 9/50
16667/16667 [==============================] - 542s 33ms/step - loss: 0.6695 - acc: 0.3644
Epoch 10/50
16667/16667 [==============================] - 542s 33ms/step - loss: 

In [42]:
# base_model_history_trainfalse = base_model_history
# combined_model_history_trainfalse = combined_model_history

In [43]:
net_weights = combined_model.get_weights()

In [44]:
# Evaluate model

### Generate color weighted

In [45]:
import re

class ColoredWeightedDoc(object):
    def __init__(self, doc, human_terms, ht_weights, token_pattern=r"(?u)\b\w\w+\b", binary = False):
        self.doc = doc
        self.human_terms = human_terms
        self.ht_weights = ht_weights
        self.binary = binary
        self.tokenizer = re.compile(token_pattern)
#         self.abs_ranges = np.linspace(0, max([abs(coefs.min()), abs(coefs.max())]), 8)
    def _repr_html_(self):
        html_rep = ""
        tokens = self.doc.split(" ") 
        if self.binary:
            seen_tokens = set()       
        for token in tokens:
            vocab_tokens = self.tokenizer.findall(token.lower())
            if len(vocab_tokens) > 0:
                vocab_token = vocab_tokens[0]
                try:
                    vocab_index = self.human_terms.index(vocab_token)
                    
                    if not self.binary or vocab_index not in seen_tokens:
                        
                        if self.ht_weights[vocab_index] == 0: # human-terms which has been washed out (opposing)
                            html_rep = html_rep + "<font size = 2, color=lightgreen> " + token + " </font>"
                        
                        elif self.ht_weights[vocab_index] != 0: # human-terms transparency
                            html_rep = html_rep + "<font size = 3, color=blue> " + token + " </font>"
                        
                        else: # neutral word
                            html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                        
                        if self.binary:    
                            seen_tokens.add(vocab_index)
                    
                    else: # if binary and this is a token we have seen before
                        html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                except: # this token does not exist in the vocabulary
                    html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
            else:
                html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
        return html_rep

In [46]:
from IPython import display
idx = 1001
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  0 
predict from base model :  [-1.] 
predict label :  [2.6941095e-08]

awful 2.4177625
boring 2.4789457
horrible 2.0174475
pathetic 1.6387914
ridiculous 2.1984637
stupid 2.1841395
waste 2.5175812


In [47]:
np.where(np.sum(y_test_agreement, axis=1)!=0)[0][1000:1010]

array([1231, 1232, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241],
      dtype=int64)

In [48]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

25000/25000 [==============================] - 7s 288us/step


[0.13369278703570367, 0.82336]

In [49]:
concat_all = concat_after_relu.predict([X_test, y_test_agreement])

In [50]:
concat_all.shape

(25000, 83)

In [51]:
indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

21874

In [52]:
not_zero_predict = combined_model.evaluate([X_test[indices], y_test_agreement[indices]], y_test_original[indices])

21874/21874 [==============================] - 6s 264us/step


In [53]:
not_zero_predict

[0.09883232563468376, 0.8788973210312891]